In [ ]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

# 회사 이름을 입력하면 해당 회사의 주식 심볼(티커)을 찾는 함수
# 예 : "Apple"을 입력하면 "AAPL" 반환

def get_ticker(inputs):
    ddg = DucDuckGoSerchAPIWrapper()
    company_name=inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")
